# When numerics goes wrong

Understanding why some numerical calculations go awry. 

Problem 5 from the homework asks for the root of $f(x) = x^3 - 2x^2 + 4x/3 - 8/27$. Let's investigate this problem with the secant method (`secantsearch` code repeated here from the previous lecture). 

In [1]:
function secantsearch(f, a::AbstractFloat, b::AbstractFloat, diagnostics=false) 
# return solution x of f(x) == 0, with a,b serving as initial guesses
    
    fa = f(a)           # cache values of f(a), f(b)     
    fb = f(b)
    
    # set c to better of a,b (and fc, too)
    c,fc = abs(fa) < abs(fb) ? (a,fa) : (b,fb)
    
    floattype = typeof((a+b)/2) # this'll eval to bigger of a,b types
    ϵ = eps(floattype)  # machine epsilon
    n = 1               # iteration count
    N = 1024            # max iterations
    
    # start vector for storing root iterates xₙ, assign x₁ = c
    x = zeros(floattype, 1)
    x[1] = c
    
    # stop when f(c) is small, |b-a| is small relative to a and b
    # or when we reach maximium number of iterations
    while abs(fc) > 10ϵ && abs(b-a)/(abs(a)+abs(b)) > 10ϵ && n < N
    
        # compute new values for c, fc by secant approximation
        c = a - fa*(b-a)/(fb-fa)
        fc = f(c)
        f′ = (fb-fa)/(b-a)
        
        # print diagnostics if they're turned on
        diagnostics && println("n = $n, c = $c, f(c) = $(f(c))")
        
        # error and exit if f(c) is bigger than both f(a) and f(b)
        if abs(fc) > abs(fa) && abs(fc) > abs(fb)
            println("secantsearch error: f(c) is bigger than f(a), f(b)")
            break # break out of while loop, don't store bad value c
        end
        
        # replace whichever of a, b has bigger f(a), f(b)
        if abs(fa) > abs(fb)
            a,fa = c,fc
        else
            b,fb = c,fc
        end      
        
        # store c in x, then increment iteration counter
        push!(x,c)   
        n += 1
    end
    
    # return c and vector x
    c, x
    
end

secantsearch (generic function with 2 methods)

In [2]:
f(x) = x^3 - 2*x^2 + 4/3*x - 8/27

f (generic function with 1 method)

In [3]:
r,x = secantsearch(f, 0.0, 1.0, true);

n = 1, c = 0.8888888888888891, f(c) = 0.010973936899862813
n = 2, c = 0.8421052631578949, f(c) = 0.005399772129616065
n = 3, c = 0.796785304247992, f(c) = 0.002203020416244428
n = 4, c = 0.7655533526628087, f(c) = 0.0009669710403885645
n = 5, c = 0.7411203540240782, f(c) = 0.0004127229640434882
n = 6, c = 0.7229262275146178, f(c) = 0.000178069284412663
n = 7, c = 0.7091194332075769, f(c) = 7.650996304064961e-5
n = 8, c = 0.6987180509624937, f(c) = 3.292610616567604e-5
n = 9, c = 0.6908601641064995, f(c) = 1.4161066591356075e-5
n = 10, c = 0.6849301974472575, f(c) = 6.091920437989451e-6
n = 11, c = 0.6804532819965489, f(c) = 2.6204324846679228e-6
n = 12, c = 0.6770739084149917, f(c) = 1.1272154389896372e-6
n = 13, c = 0.6745228512285835, f(c) = 4.848808504709368e-7
n = 14, c = 0.6725971277812471, f(c) = 2.085765059556266e-7
n = 15, c = 0.6711434384851674, f(c) = 8.972115939087644e-8
n = 16, c = 0.6700460819689645, f(c) = 3.859443598042134e-8
n = 17, c = 0.6692177117354121, f(c) = 1.6601

Hmmm, that's interesting. The secant method took 36 iterations before stopping on this $f(x)$, when it only took 7 iterations for $f(x) = x^2 - 4$. Also, `c` is still changing in the 5th digit between the last two iterations! It looks like we only have 5 or 6 digits of accuracy in our solution $r \approx 0.66667$. That decimal expansion is pretty suggestive, though. Let's check $f(2/3)$

In [4]:
f(2/3)

0.0

In [5]:
f(x) = x^3 - 2*x^2 + 4//3*x - 8//27# make rational version of f(x)

f (generic function with 1 method)

In [7]:
f(2//3)  # 2//3 is an exact root of f(x)!

0//1

Indeed, the floating-point computation of `f(2/3)` is zero, exactly! So why can't the secant search find the solution, $r=2/3$ to floating-point accuracy? i.e. `r=0.66666666666666667`?

# Investigating limits of accuracy 

To understand the difficulty of solving $f(x)=0$, let's look at $f(x)$ in the neighborhood of the root $x=2/3$. Examine $f(x+\epsilon)$ for $\epsilon = 10^{-15}$ through $10^{-1}$.

In [8]:
using Plots
gr()

Plots.GRBackend()

In [9]:
x = linspace(0, 1.2)
plot(x, f.(x), label="f(x)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 -0.2 
 
 
 -0.1 
 
 
 0.0 
 
 
 0.1 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.1436,375.813 50.1879,350.245 61.2322,326.555 72.2765,304.67 83.3208,284.52 94.3652,266.033 105.409,249.137 116.454,233.76 127.498,219.831 138.542,207.279 
 149.587,196.032 160.631,186.017 171.675,177.165 182.72,169.402 193.764,162.658 204.808,156.86 215.853,151.937 226.897,147.818 237.941,144.431 248.986,141.705 
 260.03,139.567 271.074,137.946 282.118,136.771 293.163,135.969 304.207,135.47 315.251,135.202 326.296,135.093 337.34,135.071 348.384,135.065 359.429,135.004 
 370.473,134.815 381.517,134.428 392.562,133.77 403.606,132.769 414.65,131.355 425.695,129.456 436.739,127 447.783,123.915 458.828,120.13 469.872,115.574 
 480.916,110.174 491.96,103.859 503.005,96.5579 514.049,88.1985 525.093,78.7093 536.138,68.0187 547.182,56.0552 558.226,42.7472 569.271,28.023 580.315,11.811 
 
 "/>
 
 
 
 
 f(x)

In [ ]:
ϵ = logspace(-15,-1,15)

In [ ]:
f.(2/3+ϵ)  # f.(x) means apply f to each component of x

Wha...? $f(2/3 + \epsilon)$ evaluates to zero for $\epsilon$ from $10^{-15}$ to $10^{-6}$ !!!

If  $f(2/3 + 10^{-5})$ evaluates to zero, then it's no suprise that we can't determine the $x=2/3$ root of $f(x)$ to more than five digits of accuracy.

Let's plot $f(x)$ around the root to get a sense of its behavior.

In [ ]:
using Plots
gr()
ϵ = 0.00011
x = linspace(2/3-ϵ, 2/3+ϵ, 11)
plot(x,f.(x), marker=:circ, label="f(x)")

In [26]:
ϵ = 1e-6
x = linspace(2/3-ϵ, 2/3+ϵ, 11)
@show (2/3-ϵ, 2/3+ϵ)
f.(x)

(2 / 3 - ϵ, 2 / 3 + ϵ) = (0.6666656666666666, 0.6666676666666667)


11-element Array{Float64,1}:
 -1.11022e-16
  0.0        
  0.0        
 -1.11022e-16
  0.0        
  0.0        
 -1.11022e-16
  0.0        
  0.0        
  0.0        
  0.0        

collect(x) = [0.666666, 0.666666, 0.666666, 0.666666, 0.666666, 0.666667, 0.666667, 0.666667, 0.666667, 0.666667, 0.666668]


11-element Array{Float64,1}:
 0.666666
 0.666666
 0.666666
 0.666666
 0.666666
 0.666667
 0.666667
 0.666667
 0.666667
 0.666667
 0.666668

In [18]:
g(x) = (x-2/3)^3

g (generic function with 1 method)

In [ ]:
g.(x)

In [31]:
ϵ = 1e-14
x = linspace(2/3-ϵ, 2/3+ϵ, 11)
@show (2/3-ϵ, 2/3+ϵ)
g.(x)

(2 / 3 - ϵ, 2 / 3 + ϵ) = (0.6666666666666566, 0.6666666666666766)


11-element Array{Float64,1}:
 -9.97604e-43
 -5.10773e-43
 -2.15482e-43
 -6.38467e-44
 -7.98083e-45
  0.0        
  7.98083e-45
  6.38467e-44
  2.15482e-43
  5.10773e-43
  9.97604e-43

## Question

$f(x) = x^3 - 2x^2 + 4/3 \, x - 8/27$ and $g(x) = (x-2/3)^3$ are the same function, mathematically, with the same root $r=2/3$. That is, $f(2/3) = g(2/3) = 0$. 

So why do $f(x)$ and $g(x)$ behave so differently in the neighborhood of the root, when evaluated on a computer?